# Running Agent as an Iterator (Ejecutar agente como iterador)
---

Para demostrar la funcionalidad `AgentExecutorIterator`, configuraremos un problema en el que un Agente debe:

- Recuperar tres números primos de una herramienta.
- Multiplícalos juntos.

En este simple problema podemos demostrar cómo agregar algo de lógica para verificar los pasos intermedios verificando si sus salidas son primas.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain.schema import AgentFinish
from langchain.agents.tools import Tool
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field

load_dotenv(find_dotenv())

True

In [2]:
# Para este ejemplo se necesita usar GPT-4, GPT-3.5 no lo entiende, por mucho que insistas
llm = ChatOpenAI(
    temperature=0, # Configuramos el modelo mas deterministico
    model="gpt-4", # Aqui especificamos el modelo
)

llm_math_chain = LLMMathChain.from_llm(
    llm=llm, 
    verbose=True
)

Definir herramientas (tools) que proporcionen:

- El `n` numero primo (usando un pequeño subconjunto para este ejemplo)
- El `LLMMatchChain` para actuar como calculadora

In [3]:
primes = {998: 7901, 999: 7907, 1000: 7919}


class CalculatorInput(BaseModel):
    question: str = Field()


class PrimeInput(BaseModel):
    n: int = Field()

def is_prime(n: int) -> bool:
    if n <= 1 or (n % 2 == 0 and n > 2):
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True


def get_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))


async def aget_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))


tools = [
    Tool(
        name="GetPrime",
        func=get_prime,
        description="A tool that returns the `n`th prime number",
        args_schema=PrimeInput,
        coroutine=aget_prime,
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to compute mathematical expressions",
        args_schema=CalculatorInput,
        coroutine=llm_math_chain.arun,
    ),
]

Aqui construiremos el Agente, usaremos el tipo de agente predeterminado

In [4]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

En la construccion del agente usaremos el tipo `ZERO_SHOT_REACT_DESCRIPTION` este tipo de agente que viene en Langchain usa el metodo ReAct (Reasonisns and Acting) el prompt es pequeño y consiso, no contiene muchos ejemplos (de ahi el ZERO_SHOT)

In [5]:
# Con esto se imprime el prompt que se usara en el agente.
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

GetPrime: A tool that returns the `n`th prime number
Calculator: Useful for when you need to compute mathematical expressions

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [GetPrime, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [6]:
question = "Whats is the product of the 998th, 999th and 1000th prime numbers?"

for step in agent.iter(question):
    if output := step.get("intermediate_step"):
        action, value = output[0]
        if action.tool == "GetPrime":
            print(f"Checking whether {value} is prime...")
            assert is_prime(int(value))
        # Ask user if they want to continue
        _continue = input("Should the agent continue (Y/n)?:\n")
        if _continue != "Y":
            break



> Entering new AgentExecutor chain...
I need to find the 998th, 999th, and 1000th prime numbers and then multiply them together.
Action: GetPrime
Action Input: 998
Observation: 7901
Thought:Checking whether 7901 is prime...
I have the first prime number.
Action: GetPrime
Action Input: 999
Observation: 7907
Thought:Checking whether 7907 is prime...
I have the second prime number.
Action: GetPrime
Action Input: 1000
Observation: 7919
Thought:Checking whether 7919 is prime...
I have the third prime number.
Final Answer: The product of the 998th, 999th, and 1000th prime numbers is 7901 * 7907 * 7919 = 495,219,974,633.

> Finished chain.
